In [ ]:
#CELDA 1: Explorar la estructura del dataset

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Ver la estructura de carpetas del dataset
dataset_path = '/kaggle/input/tree-counting-image-dataset'

print("📁 Estructura del dataset:\n")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Mostrar solo los primeros 5 archivos
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... y {len(files)-5} archivos más')

In [ ]:
#CELDA 2: Instalar Ultralytics (YOLOv8)

# Instalar la librería de Ultralytics que contiene YOLOv8
!pip install ultralytics -q

# Verificar la instalación
import ultralytics
from ultralytics import YOLO

print(f"✅ Ultralytics versión: {ultralytics.__version__}")
print("✅ YOLOv8 instalado correctamente")

In [ ]:
#CELDA 3: Analizar el formato de las anotaciones

# Buscar archivos de anotaciones
import json
import glob

# Buscar diferentes tipos de archivos de anotación
annotation_files = []
for ext in ['*.json', '*.xml', '*.txt', '*.csv']:
    annotation_files.extend(glob.glob(f'{dataset_path}/**/{ext}', recursive=True))

print(f"📝 Archivos de anotación encontrados: {len(annotation_files)}\n")

# Mostrar algunos ejemplos
for i, file in enumerate(annotation_files[:3]):
    print(f"{i+1}. {file}")

# Si hay archivos JSON, ver su contenido
if any(f.endswith('.json') for f in annotation_files):
    json_file = [f for f in annotation_files if f.endswith('.json')][0]
    print(f"\n📄 Contenido de ejemplo de: {os.path.basename(json_file)}")
    with open(json_file, 'r') as f:
        data = json.load(f)
        print(json.dumps(data, indent=2)[:500])  # Primeros 500 caracteres

In [ ]:
#CELDA 4: Visualizar algunas imágenes del dataset

# Buscar imágenes
image_files = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.PNG']:
    image_files.extend(glob.glob(f'{dataset_path}/**/{ext}', recursive=True))

print(f"🖼️ Total de imágenes encontradas: {len(image_files)}\n")

# Visualizar algunas imágenes
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, img_path in enumerate(image_files[:6]):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[i].imshow(img_rgb)
    axes[i].set_title(f"Imagen {i+1}\n{os.path.basename(img_path)}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print(f"\n📐 Tamaño de la primera imagen: {img.shape}")

In [ ]:
#CELDA 5: Verificar el contenido de los archivos de anotación

# Ver el contenido de un archivo de anotación para confirmar formato YOLO
label_file = '/kaggle/input/tree-counting-image-dataset/train/labels/fn_261892_4_174161_4_19_jpg.rf.8a59107bc6566dab40ca805a491abf41.txt'

print("📄 Contenido de un archivo de anotación (formato YOLO):\n")
with open(label_file, 'r') as f:
    content = f.read()
    print(content)
    
print("\n💡 Formato YOLO: <class> <x_center> <y_center> <width> <height>")
print("   Todos los valores están normalizados entre 0 y 1")

# Contar cuántas anotaciones hay por conjunto
train_labels = len(glob.glob('/kaggle/input/tree-counting-image-dataset/train/labels/*.txt'))
valid_labels = len(glob.glob('/kaggle/input/tree-counting-image-dataset/valid/labels/*.txt'))
test_labels = len(glob.glob('/kaggle/input/tree-counting-image-dataset/test/labels/*.txt'))

print(f"\n📊 Estadísticas del dataset:")
print(f"   🔵 Train: {train_labels} archivos de anotaciones")
print(f"   🟢 Valid: {valid_labels} archivos de anotaciones")
print(f"   🟡 Test: {test_labels} archivos de anotaciones")

In [ ]:
#CELDA 6: Verificar el archivo data.yaml existente

# Leer el archivo data.yaml que viene con el dataset
yaml_path = '/kaggle/input/tree-counting-image-dataset/data.yaml'

print("📄 Contenido del archivo data.yaml:\n")
with open(yaml_path, 'r') as f:
    yaml_content = f.read()
    print(yaml_content)

In [ ]:
#CELDA 8: Cargar modelo YOLOv8

# Cargar YOLOv8 
# Opciones: yolov8n.pt (nano), yolov8s.pt (small), yolov8m.pt (medium)
# Para este dataset pequeño, nano o small son suficientes

model = YOLO('yolov8n.pt')

print("✅ Modelo YOLOv8 Nano cargado")
print(f"📦 Parámetros del modelo: {sum(p.numel() for p in model.model.parameters()):,}")

In [ ]:
#CELDA 9: Entrenar el modelo

# Entrenar el modelo
results = model.train(
    data= '/kaggle/input/tree-counting-image-dataset/data.yaml',  # Usar el YAML que creamos o el original
    epochs=100,               # Número de épocas (puedes ajustar 50-150)
    imgsz=640,                # Tamaño ya es 640x640 perfecto
    batch=16,                 # Ajustar según tu GPU (8, 16, 32)
    patience=20,              # Early stopping
    save=True,                
    device=0,                 # GPU
    workers=2,                
    project='tree_detection',
    name='yolov8_trees_run1',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    lr0=0.01,                 # Learning rate
    lrf=0.01,                 # Final learning rate
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,                  # Box loss gain
    cls=0.5,                  # Class loss gain
    dfl=1.5,                  # DFL loss gain
    cos_lr=True,              # Cosine learning rate
    close_mosaic=10,          # Disable mosaic last N epochs
    amp=True,                 # Automatic Mixed Precision
    plots=True,
    verbose=True
)

print("\n🎉 ¡Entrenamiento completado!")
print(f"📁 Resultados guardados en: /kaggle/working/tree_detection/yolov8_trees_run1")

In [ ]:
#CELDA 10: Visualizar resultados del entrenamiento

from IPython.display import Image, display

# Ruta a los resultados
results_path = '/kaggle/working/tree_detection/yolov8_trees_run1'

print("📈 RESULTADOS DEL ENTRENAMIENTO\n")
print("="*60)

# Gráficos disponibles
plots = {
    'results.png': '📊 Curvas de Entrenamiento',
    'confusion_matrix.png': '🔍 Matriz de Confusión',
    'confusion_matrix_normalized.png': '🔍 Matriz de Confusión (Normalizada)',
    'F1_curve.png': '📈 Curva F1-Confidence',
    'P_curve.png': '📈 Curva Precision-Confidence',
    'R_curve.png': '📈 Curva Recall-Confidence',
    'PR_curve.png': '📈 Curva Precision-Recall',
    'val_batch0_labels.jpg': '🏷️ Etiquetas Reales (Batch Validación)',
    'val_batch0_pred.jpg': '🎯 Predicciones (Batch Validación)'
}

for plot_file, description in plots.items():
    plot_path = f'{results_path}/{plot_file}'
    if os.path.exists(plot_path):
        print(f"\n{description}")
        print("-"*60)
        display(Image(filename=plot_path, width=900))

In [ ]:
#CELDA 11: Ver métricas finales

# Cargar el mejor modelo
best_model = YOLO(f'{results_path}/weights/best.pt')

# Validar en el conjunto de validación
metrics = best_model.val(data='/kaggle/input/tree-counting-image-dataset/data.yaml', split='val')

print("="*60)
print("📊 MÉTRICAS FINALES DEL MODELO")
print("="*60)
print(f"\n🎯 mAP50 (IoU=0.5):        {metrics.box.map50:.4f}")
print(f"🎯 mAP50-95 (IoU=0.5:0.95): {metrics.box.map:.4f}")
print(f"✅ Precisión:               {metrics.box.mp:.4f}")
print(f"✅ Recall:                  {metrics.box.mr:.4f}")
print(f"⚡ F1-Score:                {2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr):.4f}")

print("\n💡 Interpretación:")
print("   • mAP50 > 0.5  = Bueno")
print("   • mAP50 > 0.7  = Muy bueno")
print("   • mAP50 > 0.9  = Excelente")

In [ ]:
#CELDA 12: Predicciones en imágenes de validación

import random

# Obtener imágenes de validación
valid_images = glob.glob('/kaggle/input/tree-counting-image-dataset/valid/images/*.jpeg')

# Seleccionar 6 imágenes aleatorias
sample_images = random.sample(valid_images, min(6, len(valid_images)))

# Hacer predicciones
results = best_model.predict(
    source=sample_images,
    conf=0.25,          # Confianza mínima 25%
    iou=0.45,           # IoU para NMS
    max_det=300,        # Máximo de detecciones
    save=False,
    verbose=False
)

# Visualizar
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, (img_path, result) in enumerate(zip(sample_images, results)):
    img_with_boxes = result.plot()
    img_rgb = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)
    
    axes[i].imshow(img_rgb)
    axes[i].set_title(
        f"🌳 {len(result.boxes)} árboles detectados\n{os.path.basename(img_path)[:30]}...",
        fontsize=10
    )
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Estadísticas
detections_per_image = [len(r.boxes) for r in results]
print(f"\n📊 Estadísticas de detección:")
print(f"   • Promedio de árboles por imagen: {np.mean(detections_per_image):.1f}")
print(f"   • Mínimo: {min(detections_per_image)} árboles")
print(f"   • Máximo: {max(detections_per_image)} árboles")

In [ ]:
#CELDA 13: Predicciones en imágenes de test

# Obtener imágenes de test
test_images = glob.glob('/kaggle/input/tree-counting-image-dataset/test/images/*.jpeg')

print(f"🧪 Evaluando en {len(test_images)} imágenes de test...\n")

# Predicciones
test_results = best_model.predict(
    source=test_images,
    conf=0.25,
    iou=0.45,
    save=True,
    project='/kaggle/working/predictions',
    name='test_predictions',
    exist_ok=True
)

# Visualizar algunas predicciones
sample_test = random.sample(list(zip(test_images, test_results)), min(6, len(test_images)))

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, (img_path, result) in enumerate(sample_test):
    img_with_boxes = result.plot()
    img_rgb = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)
    
    axes[i].imshow(img_rgb)
    axes[i].set_title(
        f"🌳 {len(result.boxes)} árboles\n{os.path.basename(img_path)[:30]}...",
        fontsize=10
    )
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ Predicciones guardadas en: /kaggle/working/predictions/test_predictions")

In [ ]:
#CELDA 14: Exportar el modelo

# Exportar a diferentes formatos
print("📦 Exportando modelo...\n")

# ONNX (recomendado para producción)
onnx_path = best_model.export(format='onnx', dynamic=False)
print(f"✅ ONNX: {onnx_path}")

# TorchScript (PyTorch)
torchscript_path = best_model.export(format='torchscript')
print(f"✅ TorchScript: {torchscript_path}")

# TensorFlow Lite (para móviles)
# tflite_path = best_model.export(format='tflite')
# print(f"✅ TFLite: {tflite_path}")

print(f"\n📍 Modelo PyTorch original: {results_path}/weights/best.pt")
print(f"📍 Último checkpoint: {results_path}/weights/last.pt")

In [ ]:
#CELDA 15: Análisis detallado del modelo

# Analizar las detecciones en todo el conjunto de validación
all_valid_results = best_model.predict(
    source='/kaggle/input/tree-counting-image-dataset/valid/images',
    conf=0.25,
    save=False,
    verbose=False
)

# Recopilar estadísticas
all_detections = []
all_confidences = []

for result in all_valid_results:
    num_trees = len(result.boxes)
    all_detections.append(num_trees)
    
    if num_trees > 0:
        confidences = result.boxes.conf.cpu().numpy()
        all_confidences.extend(confidences)

# Visualizar estadísticas
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma de detecciones por imagen
axes[0].hist(all_detections, bins=20, color='green', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Número de árboles detectados', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].set_title('📊 Distribución de Árboles por Imagen', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Histograma de confianzas
axes[1].hist(all_confidences, bins=30, color='blue', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Confianza', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].set_title('📊 Distribución de Confianza de las Detecciones', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)
axes[1].axvline(x=0.25, color='red', linestyle='--', label='Umbral (0.25)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("="*60)
print("📊 ESTADÍSTICAS GENERALES")
print("="*60)
print(f"Total de imágenes analizadas: {len(all_detections)}")
print(f"Total de árboles detectados:  {sum(all_detections)}")
print(f"Promedio de árboles/imagen:   {np.mean(all_detections):.2f}")
print(f"Desviación estándar:          {np.std(all_detections):.2f}")
print(f"Confianza promedio:           {np.mean(all_confidences):.3f}")
print(f"Confianza mínima:             {min(all_confidences):.3f}")
print(f"Confianza máxima:             {max(all_confidences):.3f}")

In [ ]:
#CELDA EXTRA A: Preparar archivos para guardar

import shutil
import os

# Crear carpeta para el output
output_dir = '/kaggle/working/model_output'
os.makedirs(output_dir, exist_ok=True)

# Copiar los archivos más importantes
print("📦 Preparando archivos para guardar...\n")

# 1. Copiar pesos del modelo
weights_dir = f'{output_dir}/weights'
os.makedirs(weights_dir, exist_ok=True)

best_model_path = '/kaggle/working/tree_detection/yolov8_trees_run1/weights/best.pt'
last_model_path = '/kaggle/working/tree_detection/yolov8_trees_run1/weights/last.pt'

shutil.copy(best_model_path, f'{weights_dir}/best.pt')
shutil.copy(last_model_path, f'{weights_dir}/last.pt')
print(f"✅ Pesos del modelo copiados")

# 2. Copiar resultados y gráficos
results_dir = f'{output_dir}/results'
os.makedirs(results_dir, exist_ok=True)

files_to_save = [
    'results.png', 
    'results.csv',
    'confusion_matrix.png',
    'F1_curve.png',
    'P_curve.png',
    'R_curve.png',
    'PR_curve.png',
    'val_batch0_pred.jpg'
]

for file in files_to_save:
    src = f'/kaggle/working/tree_detection/yolov8_trees_run1/{file}'
    if os.path.exists(src):
        shutil.copy(src, f'{results_dir}/{file}')
        print(f"✅ {file} copiado")

# 3. Copiar data.yaml
shutil.copy('/kaggle/working/data.yaml', f'{output_dir}/data.yaml')
print(f"✅ data.yaml copiado")

# 4. Guardar métricas en un archivo de texto
with open(f'{output_dir}/metrics.txt', 'w') as f:
    f.write("="*60 + "\n")
    f.write("MÉTRICAS DEL MODELO - Tree Detection YOLOv8\n")
    f.write("="*60 + "\n\n")
    f.write(f"mAP50:     {metrics.box.map50:.4f}\n")
    f.write(f"mAP50-95:  {metrics.box.map:.4f}\n")
    f.write(f"Precisión: {metrics.box.mp:.4f}\n")
    f.write(f"Recall:    {metrics.box.mr:.4f}\n")
    f.write(f"F1-Score:  {2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr):.4f}\n")

print(f"✅ Métricas guardadas")

print(f"\n🎉 Todo listo en: {output_dir}")
print("\n📁 Estructura creada:")
os.system(f'tree {output_dir} -L 2')

In [ ]:
#CELDA EXTRA B: Crear ZIP para descargar

# Crear un archivo ZIP con todo
zip_filename = '/kaggle/working/tree_detection_model.zip'

shutil.make_archive(
    '/kaggle/working/tree_detection_model',
    'zip',
    '/kaggle/working/tree_detection/yolov8_trees_run1'
)

print(f"✅ ZIP creado: {zip_filename}")
print(f"📦 Tamaño: {os.path.getsize(zip_filename) / (1024*1024):.2f} MB")
print("\n💡 Para descargar:")
print("   1. Ve a Output en el panel derecho")
print("   2. Después de 'Save Version', descarga el ZIP")

In [ ]:
#URGENTE

import shutil
import os

# Verificar que el modelo existe
model_path = '/kaggle/working/tree_detection/yolov8_trees_run1/weights/best.pt'

if os.path.exists(model_path):
    print("✅ ¡Modelo encontrado!")
    print(f"📦 Tamaño: {os.path.getsize(model_path) / (1024*1024):.2f} MB")
    
    # Copiar a una ubicación segura
    os.makedirs('/kaggle/working/SAVE_THIS', exist_ok=True)
    shutil.copy(model_path, '/kaggle/working/SAVE_THIS/best.pt')
    
    # Copiar también el último checkpoint
    shutil.copy(
        '/kaggle/working/tree_detection/yolov8_trees_run1/weights/last.pt',
        '/kaggle/working/SAVE_THIS/last.pt'
    )
    
    # Copiar data.yaml
    shutil.copy('/kaggle/working/data.yaml', '/kaggle/working/SAVE_THIS/data.yaml')
    
    # Copiar resultados importantes
    for file in ['results.png', 'confusion_matrix.png', 'results.csv']:
        src = f'/kaggle/working/tree_detection/yolov8_trees_run1/{file}'
        if os.path.exists(src):
            shutil.copy(src, f'/kaggle/working/SAVE_THIS/{file}')
    
    print("\n🎉 ARCHIVOS COPIADOS A /kaggle/working/SAVE_THIS/")
    print("\n📋 Archivos guardados:")
    for f in os.listdir('/kaggle/working/SAVE_THIS'):
        size = os.path.getsize(f'/kaggle/working/SAVE_THIS/{f}') / (1024*1024)
        print(f"   • {f} ({size:.2f} MB)")
    
else:
    print("❌ Modelo no encontrado. ¿Terminó el entrenamiento?")